#### 1. orc+zlib 和 orc+snappy的对比
ORC+Zlib after the columnar improvements no longer has the historic weaknesses of Zlib, so it is faster than SNAPPY to read, smaller than SNAPPY on disk and only ~10% slower than SNAPPY to write it out.

#### 2. spark使用orc+zlib存储hive表
```sql
create table if not exists ${distTable}(
    featureindex string,
    t1_primarykey string, 
    t1_clusterid string, 
    t1_xid string, 
    d1 map<string,string>, 
    t1_id string, 
    t1_zid string, 
    features string, 
    t2_primarykey string, 
    t2_clusterid string, 
    t2_xid string, 
    d2 map<string,string>, 
    t2_id string, 
    t2_zid string, 
    status string
)
partitioned by (save_time string)
STORED AS ORC
tblproperties ("orc.compress"="ZLIB")
```

#### 3. 查看生成的orc文件
1. 查看hdfs上的orc文件块信息  
```shell  
$ hdfs fsck hdfs://yq01-ns1/user/hive/warehouse/tmp.db/table1_hive/001319_0 -files -blocks
```

<img src="img/spark-orc-block.png">

2. 查看orc文件的stripe信息
```shell
$ hive --orcfiledump hdfs://yq01-ns1/user/hive/warehouse/tmp.db/table1_hive/001319_0 | less
```

<img src="img/spark-orc-stripe.png">